先前自己弄了一個簡單的class-Data_Loader來做訓練過程中取資料的物件，但人生苦短，何苦要重複造輪，一點都不pythonic，TensorFlow 2.0提供了`tf.data`這個便利的工具，來看看怎麼使用吧。

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'2.0.0'

指定硬體資源，相關可[參考](https://hackmd.io/@shaoeChen/ryWIV4vkL)

In [3]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')

In [4]:
gpus 

[PhysicalDevice(name=u'/physical_device:GPU:0', device_type=u'GPU'),
 PhysicalDevice(name=u'/physical_device:GPU:1', device_type=u'GPU')]

In [5]:
tf.config.experimental.set_visible_devices(devices=gpus[1], device_type='GPU')

In [6]:
tf.config.experimental.set_memory_growth(device=gpus[1], enable=True)

原本的class就不要用了，讓它隨風去，一樣取得MNIST資料集並做標準化。

In [7]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.expand_dims(x_train / 255., -1)
x_test = np.expand_dims(x_test / 255., -1)

In [8]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

將資料集與標籤做為參數提供給`tf.data`

In [9]:
datasets = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [10]:
datasets

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float64, tf.uint8)>

In [11]:
datasets = datasets.shuffle(buffer_size=128, seed=10).batch(128).repeat()

利用標準的keras Sequential來建置模型

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), padding='valid', activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), padding='valid', activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='tanh'),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(10, activation='softmax'),
])


確認模型

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               30840     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

編譯模型

In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

訓練模型

In [15]:
%%time
model.fit(datasets,
          epochs=5, 
          steps_per_epoch=int(len(x_train)/128))

Train for 468 steps
Epoch 1/5
468/468 [==============================] - 5s 10ms/step - loss: 1.0567 - accuracy: 0.7367
Epoch 2/5
468/468 [==============================] - 3s 6ms/step - loss: 0.3488 - accuracy: 0.9068
Epoch 3/5
468/468 [==============================] - 3s 6ms/step - loss: 0.2288 - accuracy: 0.9363
Epoch 4/5
468/468 [==============================] - 3s 6ms/step - loss: 0.1722 - accuracy: 0.9514
Epoch 5/5
468/468 [==============================] - 3s 6ms/step - loss: 0.1405 - accuracy: 0.9603
CPU times: user 18.9 s, sys: 2.1 s, total: 21 s
Wall time: 15.5 s


可以發現到，`model.fit`的時候我們沒有提供batch_size，因為這在datasets指定了，也沒有指定x_train與y_train，而是直接提供datasets這個物件，很方便吧。

`tf.data.Dataset`非常便利，簡單瞭解一下它有那些功能

首先是`tf.data.Dataset.prefetch`，它是一個預取機制，當GPU在訓練的時候，CPU可能正在喝杯飲料等GPU忙完叫它，等GPU訓練好一批資料之後再跟CPU說，老兄，再來一批，這時候CPU放下手邊的飲料再給GPU一批，但這中間不知不覺中浪費不少時間，如果可以跟豐田式管理一樣，都有一個預備批在那邊等，GPU拿走一批的時候CPU立馬補上一批，那中間就可以節省不少時間

In [15]:
datasets = datasets.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [16]:
%%time
model.fit(datasets,
          epochs=5, 
          steps_per_epoch=int(len(x_train)/128))

Train for 468 steps
Epoch 1/5
468/468 [==============================] - 4s 10ms/step - loss: 0.9941 - accuracy: 0.7646
Epoch 2/5
468/468 [==============================] - 2s 5ms/step - loss: 0.3315 - accuracy: 0.9108
Epoch 3/5
468/468 [==============================] - 2s 5ms/step - loss: 0.2368 - accuracy: 0.9342
Epoch 4/5
468/468 [==============================] - 2s 5ms/step - loss: 0.1856 - accuracy: 0.9475
Epoch 5/5
468/468 [==============================] - 2s 5ms/step - loss: 0.1523 - accuracy: 0.9566
CPU times: user 17.9 s, sys: 2.26 s, total: 20.1 s
Wall time: 13.1 s


上面結果可以看的出來，即使差異不大，但prefetch的時間還是快了一咪咪，`buffer_size=tf.data.experimental.AUTOTUNE`即讓框架自己決定要預取多少資料，更多的資訊可參考官方[Optimize pipeline performance](https://www.tensorflow.org/guide/data_performance)說明

為了能夠說明，我們重新取資料，這次取資料並不先做資料標準化，只是單純的先多推一個軸

In [52]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(dtype='float32')
x_test = x_test.astype(dtype='float32')
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [53]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [54]:
x_train[0].mean()

35.108418

`tf.data.Dataset.from_tensor_slices`，由函數名稱不難看出，是一種取得資料的函數，來源需要是tensor

下面範例意味著將我們取得的MNIST資料餵給`tf.data.Dataset`

In [55]:
datasets = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [56]:
for _x in  datasets.take(1):
    print(_x[0].numpy().mean())

35.108418


接著我們希望可以將資料標準化或做某些前置預處理，也許原本的放入Dataset的資料是檔案路徑，我們需要利用路徑來載入照片轉為numpy再轉為tensor，因此我們需要設置一個函數

這個函數很簡單，只是為了做標準化而以，當然實際上不會這麼做，是為了範例才這麼做

In [57]:
def prepare(x_train_s, y_label_s):
    x_train_s = x_train_s / 255.
    return x_train_s, y_label_s

接下來我們就可以利用`tf.data.Dataset.map`來綁定標準化函數

In [58]:
datasets = datasets.map(map_func=prepare, 
                        num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [59]:
for _x in  datasets.take(1):
    print(_x[0].numpy().mean())

0.13768007


可以發現到，我們的資料集已經標準化完成了

訓練過程中的資料一定要打散，這可以透過`tf.data.Dataset.shuffle`來實作

In [60]:
datasets = datasets.shuffle(buffer_size=10000)

In [61]:
for _x in  datasets.take(1):
    print(_x[0].numpy().mean())

0.16185975


資料已經亂掉了，要注意到的是，buffer_size設置愈小就愈不亂，代表設置1就是沒有洗牌，設置整個資料集數量就是整個大洗牌。舉例來說，buffer_size為10000，代表會從資料集拿10000筆資料先放到一個緩衝區去，然後再把這10000筆資料亂塞回去這原本的10000坑洞，因此如果設置1就是什麼都沒有交換了。

最後，我們希望定義每一個batch的大小，避免訓練過程中是整個資料集塞進去GPU

In [62]:
datasets = datasets.batch(128, drop_remainder=False)

這邊參數`drop_remainder`指的是，當資料集大小/batch_size無法整除的時候是否放棄那不成批的資料

最後不要忘了設置prefetch來增加效能

In [64]:
datasets = datasets.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

現在就可以開始訓練了

In [65]:
%%time
model.fit(datasets,
          epochs=5, 
          steps_per_epoch=int(len(x_train)/128))

Train for 468 steps
Epoch 1/5
468/468 [==============================] - 2s 4ms/step - loss: 0.1279 - accuracy: 0.9631
Epoch 2/5
  1/468 [..............................] - ETA: 3s - loss: 0.0604 - accuracy: 1.0000WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 2340 batches). You may need to use the repeat() function when building your dataset.
CPU times: user 7.03 s, sys: 2.03 s, total: 9.06 s
Wall time: 2.1 s


怎麼會這樣?我明明設置5個epoch，結果一個半就停了?這是因為，我們的資料集就是只有那麼多，迭代完了自然就停止了，因此最後還要設置一個`tf.data.Dataset.repeat`

In [66]:
datasets = datasets.repeat(count=-1)

設置`count=-1`就代表天長地久，不會停止

In [67]:
%%time
model.fit(datasets,
          epochs=5, 
          steps_per_epoch=int(len(x_train)/128))

Train for 468 steps
Epoch 1/5
468/468 [==============================] - 2s 4ms/step - loss: 0.1106 - accuracy: 0.9677
Epoch 2/5
468/468 [==============================] - 2s 4ms/step - loss: 0.0975 - accuracy: 0.9717
Epoch 3/5
468/468 [==============================] - 2s 4ms/step - loss: 0.0874 - accuracy: 0.9742
Epoch 4/5
468/468 [==============================] - 2s 4ms/step - loss: 0.0789 - accuracy: 0.9770
Epoch 5/5
468/468 [==============================] - 2s 4ms/step - loss: 0.0724 - accuracy: 0.9789
CPU times: user 34.3 s, sys: 9.64 s, total: 44 s
Wall time: 8.96 s
